# 🐦 BIRD-SQL Evaluation with Google BigQuery

In [ ]:
# STEP 1: Install dependencies
!pip install -q datasets pandas tqdm sqlparse openai google-cloud-bigquery


In [ ]:
# STEP 2: Load BIRD dataset
from datasets import load_dataset

dataset = load_dataset("Yale-LILY/BIRD", split="test")
dataset = dataset.select(range(5))  # Sample 5 rows for testing
dataset[0]


In [ ]:
# STEP 3: Load schema from tables.json
import json

!wget https://raw.githubusercontent.com/Yale-LILY/BIRD/main/database/tables.json -O tables.json

with open("tables.json") as f:
    tables = json.load(f)

# Get schema for a db_id
db_id = dataset[0]["db_id"]
schema = next(t for t in tables if t["db_id"] == db_id)

def format_schema(schema):
    table_names = schema["table_names_original"]
    columns = schema["column_names_original"]
    column_text = []
    for idx, (table_idx, col_name) in enumerate(columns):
        if col_name == "*":
            continue
        table_name = table_names[table_idx]
        column_text.append(f"{table_name}.{col_name}")
    return " | ".join(column_text)

formatted_schema = format_schema(schema)
print("Schema preview:", formatted_schema)


In [ ]:
# STEP 4: Generate SQL using OpenAI
import openai

openai.api_key = "sk-..."  # Replace with your OpenAI key

def generate_sql(question, schema_text):
    prompt = f"""Given the table schema:
{schema_text}

Convert this natural language question to a SQL query:
"{question}"

SQL:"""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"ERROR: {str(e)}"


In [ ]:
# STEP 5: Generate SQL for each question
samples = []
for item in dataset:
    q = item["question"]
    db_id = item["db_id"]
    schema = next(t for t in tables if t["db_id"] == db_id)
    schema_text = format_schema(schema)
    sql = generate_sql(q, schema_text)
    print(f"Q: {q}\nSQL: {sql}\n")
    samples.append({"question": q, "sql": sql, "db_id": db_id})


In [ ]:
# STEP 6: Execute SQL on BigQuery
from google.cloud import bigquery

client = bigquery.Client()
project_id = "your-project-id"
dataset_id = "bird_dataset"  # Replace with your dataset name

def run_sql_on_bigquery(sql):
    try:
        job = client.query(sql)
        return [dict(row) for row in job.result()]
    except Exception as e:
        return {"error": str(e)}

# Example run (you can loop through all)
for s in samples:
    result = run_sql_on_bigquery(s["sql"])
    print(f"SQL:\n{s['sql']}\nResult:\n{result}\n{'='*40}")


In [ ]:
# STEP 7: Save results
with open("generated_results.jsonl", "w") as f:
    for s in samples:
        f.write(json.dumps(s) + "\n")

print("Saved to generated_results.jsonl")


## 📌 Example: BigQuery CREATE TABLE SQL and CSV Upload

In [ ]:
# Sample CREATE TABLE (adjust types as needed)
create_sql = """
CREATE TABLE `your-project.bird_dataset.department` (
    Department_ID STRING,
    Name STRING,
    Location STRING
);
"""

# Load CSV into BigQuery
from google.cloud import bigquery

table_id = "your-project.bird_dataset.department"
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True)

with open("department.csv", "rb") as source_file:
    load_job = client.load_table_from_file(source_file, table_id, job_config=job_config)
load_job.result()
print("Loaded data into BigQuery table:", table_id)
